This week for discussion section we will discuss some problems, including
some from HW 6.

## Review

Recall that in lecture we learned we can do the following things:
- DFS/BFS: For example, given vertices $u, v$ in a graph $(V, E),$ we can
determine whether there exists a path from $u$ to $v$ in $O(|V| + |E|)$ time.
- Dikjstra's algorithm (using a binary heap): Given vertices $u, v$ in a
connected weighted undirected graph $(V, E, W),$ find the length of the
shortest (with respect to $W$) path from $u$ to $v$ in $O(|E| \log |V|)$ time.
- Topological sort: Given a directed acyclic graph $(V, E),$ find an
ordering $v_1, \dotsc, v_n \in V$ of the vertices such that $(v_i, v_j) \in E$
only if $v_i$ comes before $v_j$ in the ordering.

Here are implementations of Dikjstra and topological sort, following Wikipedia.

In [8]:
import heapq
import copy

class WeightedUndirectedGraph:

    def __init__(self, n_vertices, edges):
        self.vertices = list(range(n_vertices))
        self.edges = [[] for _ in self.vertices]
        for u, v, weight in edges:
            self.edges[u].append((v, weight))
            self.edges[v].append((u, weight))
        
    def shortest_path(self, s, t):
        dist = [0 if u == s else float('inf') for u in self.vertices]
        heap = []
        heapq.heappush(heap, (s, 0))
        while heap:
            u, d = heapq.heappop(heap)
            for v, weight in self.edges[u]:
                alt = dist[u] + weight
                if alt < dist[v]:
                    dist[v] = alt
                    heapq.heappush(heap, (v, alt))
        return dist[t]


G = WeightedUndirectedGraph(
    7,
    [
        (1, 2, 7),
        (1, 3, 9),
        (1, 6, 14),
        (2, 3, 10),
        (2, 4, 15),
        (3, 4, 11),
        (3, 6, 2),
        (4, 5, 6), 
        (5, 6, 9)
    ]
)
assert G.shortest_path(1, 5) == 20
assert G.shortest_path(1, 4) == 20
assert G.shortest_path(2, 6) == 12
assert G.shortest_path(4, 5) == 6
assert G.shortest_path(3, 5) == 11


class DirectedAcyclicGraph:

    def __init__(self, n_vertices, edges):
        self.vertices = list(range(n_vertices))
        self.edges = [[] for _ in self.vertices]
        for u, v in edges:
            self.edges[u].append(v)
    
    def topological_sort(self):
        dag = copy.deepcopy(self)
        no_incoming_edge = lambda u: all(
            u not in dag.edges[v] for v in dag.vertices
        )
        l = []
        S = list(filter(no_incoming_edge, dag.vertices))
        while S:
            n = S.pop()
            l.append(n)
            while dag.edges[n]:
                m = dag.edges[n].pop()
                if no_incoming_edge(m):
                    S.append(m)
        return l

dag = DirectedAcyclicGraph(
    7,
    [(0, 1), (0, 2), (1, 2), (1, 5), (2, 3), (5, 3), (5, 4), (6, 1), (6, 5)]
)
top_sort = dag.topological_sort()
assert all(
    0 <= top_sort.index(u) < top_sort.index(v)
    for u in dag.vertices
    for v in dag.edges[u]
)

## Problem 2

Let us start with a thought experiment. Let $G = (V, E)$ be a DAG and let
$v_1, \dotsc, v_n$ be a topological sort of its vertices. The paths in $G$ are
in bijection with the subsequences $v_{i_1}, \dotsc, v_{i_k}$ of the topological
sort such that $(v_{i_j}, v_{i_{j + 1}}) \in E$ for $j = 1, \dotsc, k - 1.$
(Think about this carefully. You will need the use the assumption that $G$ is
acyclic and also the definition of a topological sort.) To solve Problem 2,
you should think about the following: what do length $n$ paths and length $n$
subsequences look like?

## Course Prerequisites Problem

Suppose you need to take courses $1, \dotsc, n$ and that course $i$ takes
$t[i]$ time to complete. There are prerequisites
$(r_1, s_1), \dotsc, (r_m, s_m),$ in the sense that course $r_i$ must be taken
before $s_i,$ but it is possible to complete all the courses, e.g. there will
not be two courses that directly depend on each other. How fast can you complete
all the courses?

#### Solution

Consider the directed graph $G = (V, E)$ whose vertices are the courses
$1, \dotsc, n$ and whose edges are $(r_1, s_1), \dotsc, (r_m, s_m).$ Since it
is possible to complete all the courses, $G$ is acyclic. Thus, let
$v_1, \dotsc, v_n$ be a topological sort of the vertices of $G.$ Denoting by
$f(j)$ the minimum time to complete courses $v_1, \dotsc, v_j,$ we therefore
have the recurrence
$$f(j + 1) = t[j + 1] + \max(f(i) \text{ for } i = 1, \dotsc, j \text{ such that } (v_i, v_j))$$
with base case $f(1) = t[v_1].$ Our answer is then $f(n).$

In [ ]:
def courses(n, t, prereqs):
    dag = DirectedAcyclicGraph(
        n,
        [(r - 1, s - 1) for r, s in prereqs]
    )
    top_sort = dag.topological_sort()
    cache = n * [0]
    cache[0] = t[top_sort[0]]
    for j in range(1, n):
        cache[j] = t[top_sort[j]] + max(
            [
                cache[i]
                for i in range(j)
                if top_sort[j] in dag.edges[top_sort[i]]
            ] + [0]
        )
    return cache[n - 1]

assert courses(3, [3, 2, 5], [(1, 3), (2, 3)]) == 8
assert courses(
    5,
    [1, 2, 3, 4, 5],
    [(1, 5), (2, 5), (3, 5), (3, 4), (4, 5)]
) == 12

## Running theme

Problems 1, 3, and 4 have a running theme of constructing auxilliary graphs.
This helps not only in inventing solutions but also in proving that they
are correct. Let us discuss two artificial examples of this:
- What if we modify Problem 2 to ask that we not visit some given vertices
$u_1, \dotsc, u_m$? Then we can simply delete them from $G,$ resulting in the
auxilliary graph $G',$ and then run our original algorithm on $G'.$
- What if we modify Problem 2 to ask that we visit a special vertex $v$ exactly
$10$ times? Then we can construct an auxilliary graph $G'$ by creating $9$
extra copies of $v$ and the edges coming into and out of $v.$ Now a path that
visits every vertex in $G'$ exactly once is a path in $G$ that visits $v$
exactly $10$ times and every other vertex exactly once. Thus we simply run our
original algorithm on $G'.$

Let us discuss a nontrivial example of this.

## Avoidant schedules problem

(This is Solved Exercise 2 in Chapter 3 of the textbook.) Let $G = (V, E)$ be a
graph. Suppose Alice and Bob are initially at vertices $a, b \in V$ and want to
get to vertices $c, d \in V$ respectively. Only one of them can move at a time,
and they cannot ever be within a distance $5$ from each other. Can they agree
on a schedule so they both get to their destinations?

#### Solution

Consider the auxilliary graph $G'$ that tracks all pairs of configurations of
Alice and Bob as well as the possible moves: the vertices are $V \times V,$ and
the edges are those of the form $((x, u), (x, v))$ or $((u, x), (v, x))$ for
$x \in V$ and $(u, v) \in E.$ To impose that they always stay more than $5$
distance apart, we take the full subgraph $G''$ of vertices $(u, v)$ where $u$
and $v$ are a distance more than $5$ apart in $G.$ The problem now becomes
finding a path from $(a, b)$ to $(c, d)$ (they exist WLOG) in our
auxilliary-auxilliary graph $G''.$

Let us analyze the runtime of this. Constructing $G'$ takes $O(|V|^2 + |V||E|)$
time because there are $|V|^2$ vertices and $|V||E|$ edges. We can construct
$G''$ in $O(|V|(|V| + |E|))$ by DFS'ing (which takes $O(|V| + |E|)$ time) from
each vertex in $G.$ Finally, determining whether there is a path from $(a, b)$
to $(c, d)$ in $G''$ can be done with a DFS, so it takes $O(|V|^2 + |V||E|)$
time as well. We conclude that the overall runtime is $O(|V|^2 + |V||E|).$

## Problem 3

Let us discuss one last problem, also related to the theme of constructing
auxilliary graphs. Suppose $G = (V, E)$ is a graph where each edge
$(u, v) \in E$ has a color $c(u, v)$ that is either red or blue. Given two
vertices $s, t \in V$ determine whether there is an alternating red-blue path
from $s$ to $t.$

#### Solution

Let us make two copies of $G,$ a lower one and a higher one, and let us imagine
that red edges point downward and blue edges point upward. More precisely,
consider the auxilliary directed graph $G'$ whose vertices are
$(V \times \{-1\}) \cup (V \times \{+1\})$ and whose edges are
$((u, 1), (v, -1))$ for $(u, v) \in E$ whose  $c(u, v)$ is red and
$((u, -1), (v, 1))$ for $(u, v) \in E$ whose $c(u, v)$ is blue. Now the paths in
$G'$ are the alternating paths in $G.$ We can now carefully translate what the
problem is asking for into our graph $G'.$